In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import leaguedashplayerstats

data_DIR = "../fdata/"
fig_DIR_p = "../figs/player_leaders/"
fig_DIR_t = "../figs/team_leaders/"

In [ ]:
year = []
pts_40 = []
for season in range(2000,2023):
    df = pd.read_parquet(data_DIR + "NBA" + "_Player_BoxScores_" + "Indv" + "_" + str(season) + ".parquet")
    pts_40_1 = df.query("PTS>=50").PTS.count()
    year.append(season+1)
    pts_40.append(pts_40_1)

In [ ]:
df = pd.DataFrame({"Season":year,"PTS_40":pts_40})

In [ ]:
df